In [2]:
import pandas as pd
import datetime
import time
import requests
import numpy as np

In [86]:
cd = "122630"

list_df = []
for pg in range(2):
    refer_url = f"https://finance.naver.com/item/sise_day.naver?code=122630&page={pg+1}"
    headers = {
        "referer" : refer_url,    
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
    }

    res = requests.get(url=refer_url, headers=headers)
    list_df.append(pd.read_html(res.text)[0].dropna())

df_base = pd.concat(list_df)
df_base = df_base.drop_duplicates()
df_base = df_base.sort_values(by=["날짜"], ascending=False)
df_base["GAP"] = df_base["고가"] - df_base["저가"]
df_base["DIFF"] = df_base["종가"].diff()
df_base

,날짜,종가,전일비,시가,고가,저가,거래량,GAP,DIFF
1,2022.04.13,19440.0,505.0,19075.0,19440.0,18990.0,10930183.0,450.0,NaN
2,2022.04.12,18935.0,365.0,19020.0,19195.0,18790.0,19387013.0,405.0,-505.0
3,2022.04.11,19300.0,25.0,19105.0,19460.0,19015.0,12602548.0,445.0,365.0
4,2022.04.08,19325.0,45.0,19355.0,19405.0,19085.0,15956735.0,320.0,25.0
5,2022.04.07,19280.0,560.0,19530.0,19590.0,19200.0,17045371.0,390.0,-45.0
9,2022.04.06,19840.0,440.0,19975.0,20055.0,19810.0,14118743.0,245.0,560.0
10,2022.04.05,20280.0,25.0,20445.0,20510.0,20150.0,11652397.0,360.0,440.0
11,2022.04.04,20305.0,200.0,19930.0,20325.0,19810.0,10471018.0,515.0,25.0
12,2022.04.01,20105.0,285.0,20165.0,20210.0,19905.0,11811809.0,305.0,-200.0
13,2022.03.31,20390.0,135.0,20215.0,20580.0,20210.0,15020402.0,370.0,285.0


In [65]:
list_end_prc = np.array(df_base["종가"])
list_cprice = np.array(df_base["시가"])
list_hprice = np.array(df_base["고가"])
list_lprice = np.array(df_base["저가"])
list_diff = np.array(df_base["DIFF"])[1:]

In [66]:
np.median(list_end_prc), np.median(list_cprice), np.median(list_hprice), np.median(list_lprice)

(19937.5, 20067.5, 20187.5, 19765.0)

In [67]:
[np.quantile(list_end_prc, .25), np.quantile(list_cprice, .25), np.quantile(list_lprice, .25), np.quantile(list_hprice, .25)]

[19335.0, 19466.25, 19143.75, 19786.25]

In [68]:
np.quantile(list_lprice, .25)

19143.75

In [77]:
list_diff

array([  985.,   350.,   480.,  -405.,    90.,   565.,   330.,  -370.,
        -575.,  -245.,   720.,   265.,  -155.,    20.,  -600.,   150.,
       -1095.,   430.,   280.,  -105.,   700.,  -605.,  -935.,  -465.,
         800.,  -175.,   -55.,  -345.,   540.,   570.,   100.,  -405.,
         350.,   325.,  -180.,  -135.,    25.,   175.,   140.,   135.,
        -285.,   200.,   -25.,  -440.,  -560.,    45.,   -25.,  -365.,
         430.])

In [84]:
inc_seq = 0
dec_seq = 0
inc_max = 0
dec_max = 0

for num in list_diff:
    if num > 0:
        inc_seq += 1
        if inc_seq > inc_max:
            inc_max = inc_seq
        dec_seq = 0
    elif num < 0:
        dec_seq += 1
        if dec_seq > dec_max:
            dec_max = dec_seq
        inc_seq = 0
    else: continue
    
inc_max, dec_max

(4, 3)

In [61]:
np.median(list_diff[list_diff > 0]), np.median(list_diff[list_diff < 0])

(157.5, -285.0)

In [62]:
# 가장 최근 몇번의 증감이 있었는지...
list_dec = list_diff[list_diff < 0]
list_inc = list_diff[list_diff > 0]

list_dec, list_inc

(array([-405., -180., -135., -285.,  -25., -440., -560.,  -25., -365.]),
 array([100., 350., 325.,  25., 175., 140., 135., 200.,  45., 410.]))

In [110]:
# 기준가를 지정
# 연속 3틱 두번 발생하면 매수 - 상승세로 판단.
# 연속 5틱 두번 발생하면 매수 - 반등할거라 판단.
# 꺽이는 지점에서 기준가 변경
base_prc = 100
list_prc = np.random.randint(18900, 19500, size=100)

In [111]:
list_prc

array([19446, 19408, 19132, 19219, 19030, 19164, 19490, 19354, 19363,
       19161, 19493, 19273, 19376, 19373, 19242, 19132, 18927, 19041,
       18906, 19227, 18945, 19082, 19419, 19338, 19258, 19438, 19363,
       18922, 19361, 19285, 19496, 19097, 19220, 18982, 19049, 19113,
       19156, 19164, 18930, 19270, 19345, 18989, 19495, 19212, 19194,
       19445, 18993, 19181, 19103, 19356, 18909, 18961, 19378, 19170,
       19055, 19307, 18939, 18991, 18928, 19186, 19179, 19160, 18985,
       19106, 19089, 19406, 19452, 19198, 19335, 19317, 19306, 19062,
       19348, 18938, 19116, 19189, 18953, 18998, 19070, 18912, 19177,
       19087, 19013, 19251, 19158, 19283, 19064, 19229, 19059, 19216,
       19082, 19270, 19074, 19046, 19083, 18964, 19084, 19160, 19228,
       19354])